# Implementing and training attention mechanism

In [1]:
import random
import tensorflow as tf
import pandas as pd
# Korean NLP package (Open Korean Tweet Parser)
from konlpy.tag import Okt

In [2]:
EPOCHS = 50
NUM_WORDS = 2000

## Encoder

In [3]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_state=True)
        
    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        _, h, c = self.lstm(x)
        return h, c

## Decoder

In [4]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        # Adding layers for shifted output
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        # decoder need to store whole state from lstm so return_sequences should be True
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')
        
    def call(self, inputs, training=False, mask=None):
        x, h, c = inputs
        x = self.emb(x)
        x, h, c = self.lstm(x, initial_state=[h, c])
        return self.dense(x), h, c

## Seq2Seq

In [5]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos
        
    def call(self, inputs, training=False, mask=None):
        # When training, input is just shifted, so the process is very simple
        if training:
            x, y = inputs
            h, c = self.enc(x)
            y, _, _ = self.dec((y, h, c))
            return y
        else:
            x = inputs
            h, c = self.enc(x)
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))
            
            seq = tf.TensorArray(tf.int32, 64)
            
            # Through tf.range, we can iterate it with autoGraph
            for idx in tf.range(64):
                y, h, c = self.dec([y, h, c])
                
                # To convert sparse type
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                
                # To express batch shape
                y = tf.reshape(y, (1, 1))
                seq = seq.write(idx, y)
                
                if y == self.eos:
                    break
                
            return tf.reshape(seq.stack(), (1, 64))

## Training Loop

In [6]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]
    
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)
    
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## Preparing Dataset

In [3]:
okt = Okt()

chatbot_data = pd.read_csv('./dataset/ChatbotData.csv')
chatbot_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [4]:
questions = [' '.join(okt.morphs(line)) for line in chatbot_data['Q']]
answers = [' '.join(okt.morphs(line)) for line in chatbot_data['A']]

In [5]:
answers_t = ['\t ' + line + ' \n' for line in answers]

In [6]:
answers_t

['\t 하루 가 또 가네요 . \n',
 '\t 위로 해 드립니다 . \n',
 '\t 여행 은 언제나 좋죠 . \n',
 '\t 여행 은 언제나 좋죠 . \n',
 '\t 눈살 이 찌푸려지죠 . \n',
 '\t 다시 새로 사는 게 마음 편해요 . \n',
 '\t 다시 새로 사는 게 마음 편해요 . \n',
 '\t 잘 모르고 있을 수도 있어요 . \n',
 '\t 시간 을 정 하고 해보세요 . \n',
 '\t 시간 을 정 하고 해보세요 . \n',
 '\t 자랑 하는 자리 니까 요 . \n',
 '\t 그 사람 도 그럴 거 예요 . \n',
 '\t 그 사람 도 그럴 거 예요 . \n',
 '\t 혼자 를 즐기세요 . \n',
 '\t 돈 은 다시 들어올 거 예요 . \n',
 '\t 땀 을 식혀주세요 . \n',
 '\t 어서 잊고 새 출발 하세요 . \n',
 '\t 빨리 집 에 돌아가서 끄고 나오세요 . \n',
 '\t 빨리 집 에 돌아가서 끄고 나오세요 . \n',
 '\t 다음 달 에는 더 절약 해봐요 . \n',
 '\t 따뜻하게 사세요 ! \n',
 '\t 다음 달 에는 더 절약 해봐요 . \n',
 '\t 가장 확실한 시간 은 오늘이 에요 . 어제 와 내일 을 놓고 고민 하느라 시간 을 낭비하지 마세요 . \n',
 '\t 온 가족 이 모두 마음 에 드는 곳 으로 가보세요 . \n',
 '\t 온 가족 이 모두 마음 에 드는 곳 으로 가보세요 . \n',
 '\t 온 가족 이 모두 마음 에 드는 곳 으로 가보세요 . \n',
 '\t 저 를 만들어 준 사람 을 부모님 , 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요 \n',
 '\t 저 를 만들어 준 사람 을 부모님 , 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요 \n',
 '\t 더 가까워질 기회 가 되겠네요 . \n',
 '\t 저 도 요 . \n',
 '\t 다 들 바빠서 이야기 할 시간 이 부족했나 봐요 . \n',
 '\t 다 들 바빠서 이야기 할 

In [7]:
num_sample = len(questions)

In [8]:
perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

In [9]:
train_q = []
train_a = []
test_q = []
test_a = []

for idx, qna in enumerate(zip(questions, answers_t)):
    q, a = qna
    if perm[idx] > num_sample // 5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)
    

In [10]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                 filters='!"#$%()*+,-./:;<=>?@[\\]^_`{|}~')

In [11]:
tokenizer.fit_on_texts(train_q + train_a)

In [12]:
train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

In [13]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq, 
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)

y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post', 
                                                        maxlen=65)

X_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre', 
                                                       maxlen=64)

y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq, 
                                                       value=0, 
                                                       padding='post', 
                                                       maxlen=65)

In [14]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000)\
                                                                .batch(32).prefetch(1024)

test_ds  = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(1).prefetch(1024)

## Define training process
### Create model, loss function, optimizer, and metrics

In [19]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'], eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [20]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1, 
                          train_loss.result(), 
                          train_accuracy.result() * 100))
    
    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 0.6210343241691589, Accuracy: 91.7951431274414
Epoch 2, Loss: 0.4339092969894409, Accuracy: 93.10025024414062
Epoch 3, Loss: 0.40634220838546753, Accuracy: 93.39448547363281
Epoch 4, Loss: 0.38566476106643677, Accuracy: 93.612548828125
Epoch 5, Loss: 0.3703303337097168, Accuracy: 93.75066375732422
Epoch 6, Loss: 0.3564894497394562, Accuracy: 93.87158966064453
Epoch 7, Loss: 0.3434694707393646, Accuracy: 94.00457763671875
Epoch 8, Loss: 0.3314449191093445, Accuracy: 94.10948181152344
Epoch 9, Loss: 0.31972014904022217, Accuracy: 94.22859191894531
Epoch 10, Loss: 0.30857598781585693, Accuracy: 94.34539794921875
Epoch 11, Loss: 0.29714226722717285, Accuracy: 94.47821807861328
Epoch 12, Loss: 0.2862580418586731, Accuracy: 94.59864807128906
Epoch 13, Loss: 0.2753060460090637, Accuracy: 94.74287414550781
Epoch 14, Loss: 0.26440250873565674, Accuracy: 94.86925506591797
Epoch 15, Loss: 0.2534867227077484, Accuracy: 95.02091217041016
Epoch 16, Loss: 0.24267737567424774, Accuracy:

KeyboardInterrupt: 

## Test process

In [21]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('Q: ', test_text)
    print('A: ', gt_text)
    print('p: ', texts)

_
Q:  ['1 학교 떨어졌어']
A:  ['\t 위로 해 \n']
p:  ['저 도 모르겠어요 \n']
_
Q:  ['']
A:  ['\t 이 \n']
p:  ['\n']
_
Q:  ['']
A:  ['\t 돈 은 다시 거 예요 \n']
p:  ['\n']
_
Q:  ['']
A:  ['\t 어서 잊고 새 출발 하세요 \n']
p:  ['\n']
_
Q:  ['가장 확실한 건 뭘 까']
A:  ['\t 가장 확실한 시간 은 오늘이 에요 어제 와 내일 을 놓고 고민 하느라 시간 을 마세요 \n']
p:  ['가장 확실한 시간 은 오늘이 에요 어제 와 내일 을 놓고 고민 하느라 시간 을 마세요 \n']
_
Q:  ['가족 여행 가기 로 했어']
A:  ['\t 온 가족 이 모두 마음 에 드는 곳 으로 가보세요 \n']
p:  ['말 하기 좋은 계절 이네 요 \n']
_
Q:  ['가족 들 이랑 어디 가지']
A:  ['\t 온 가족 이 모두 마음 에 드는 곳 으로 가보세요 \n']
p:  ['발 이 \n']
_
Q:  ['강아지 싶어']
A:  ['\t 수 있을 때 보세요 \n']
p:  ['여행 은 \n']
_
Q:  ['같이 게임 해도 되나']
A:  ['\t 안 될 것 도 없죠 \n']
p:  ['데이트 신청 해보세요 \n']
_
Q:  ['같이 나 만 거 같아']
A:  ['\t 연인 은 잘 못 하고 않을 거 예요 \n']
p:  ['마음 이라도 할 수 있는 걸 찾아보세요 \n']
_
Q:  ['같이 할 수 있는 취미 생활 뭐 있을까']
A:  ['\t 함께 하면 서로 를 더 많이 알 게 될 거 예요 \n']
p:  ['그 분 이 무엇 을 좋아하는지 \n']
_
Q:  ['하 니까 좋다']
A:  ['\t 친구 들 이 봐요 \n']
p:  ['마음 의 안정 을 기 좋은 취미 네 요 \n']
_
Q:  ['거짓말 을 나도 자꾸 해']
A:  ['\t 거짓말 은 할수록 \n']
p:  ['더 표현 해 달라 고 말 해보세요 \n']
_
Q:  ['거짓말 을 

_
Q:  ['나 랑 있는게 봐']
A:  ['\t 상대방 을 이해해 주세요 \n']
p:  ['사랑 은 하고 \n']
_
Q:  ['']
A:  ['\t 아무 것 도 안해 도 괜찮아요 \n']
p:  ['\n']
_
Q:  ['나를 너무 오래 했어']
A:  ['\t 기다리는 동안 많은 생각 이 \n']
p:  ['매력 을 해보세요 \n']
_
Q:  ['나를 수 있는 건 뭐 가 있을까']
A:  ['\t 지금 모습 도 좋아요 \n']
p:  ['제 가 있잖아요 \n']
_
Q:  ['나를 로 아는 사람 어떡해']
A:  ['\t 하지 마세요 \n']
p:  ['무시 하세요 \n']
_
Q:  ['나 만 반 친구 없어']
A:  ['\t 친구 를 사귈 수 있을 거 예요 \n']
p:  ['맞았나 봐요 \n']
_
Q:  ['나 만 거야']
A:  ['\t 그 사람 도 거 예요 \n']
p:  ['울 어도 괜찮아요 \n']
_
Q:  ['나 만 봐']
A:  ['\t 와 대화 를 나눠 보세요 \n']
p:  ['누군가 를 아니죠 \n']
_
Q:  ['나 만 야근 해']
A:  ['\t 얼른 기도 할게요 \n']
p:  ['다음 부터는 잘 \n']
_
Q:  ['나 만 거 같아']
A:  ['\t 스스로 \n']
p:  ['선생님 께 도움 을 해보세요 \n']
_
Q:  ['나 만 힘든 거']
A:  ['\t 누구 나 힘들어요 \n']
p:  ['마음 이 \n']
_
Q:  ['나중 에 뭐 하고 먹고']
A:  ['\t 진짜 하고 싶은 걸 찾아보세요 \n']
p:  ['이 될 때 아무렇지도 않으면 거 예요 \n']
_
Q:  ['']
A:  ['\t 한 번 헤어 나올 수 없다고 해 요 \n']
p:  ['\n']
_
Q:  ['비 추워']
A:  ['\t 따뜻하게 사세요 \n']
p:  ['오늘 래 요 \n']
_
Q:  ['이 안 추워']
A:  ['\t 보일러 가 으로 이 되는지 보세요 \n']
p:  ['어서 따듯 한 곳 으로 가세 요 \n']
_
Q: 

_
Q:  ['되고 싶은 게 두']
A:  ['\t 더 괜찮아요 \n']
p:  ['\n']
_
Q:  ['드디어 개강 이다']
A:  ['\t 곧 방학 이 예요 \n']
p:  ['기억 으로 거 예요 \n']
_
Q:  ['드디어 손']
A:  ['\t \n']
p:  ['\n']
_
Q:  ['드디어 커플 반지 받았어']
A:  ['\t 로맨틱 한 선물 이네 요 \n']
p:  ['당황 했겠어요 \n']
_
Q:  ['드디어 여행 간다']
A:  ['\t \n']
p:  ['좋은 습관 이에요 \n']
_
Q:  ['잘 안되네']
A:  ['\t 을 연습 \n']
p:  ['잘 좋겠네요 \n']
_
Q:  ['가 잘 안 돼']
A:  ['\t 을 연습 \n']
p:  ['자연스러운 현상 이에요 \n']
_
Q:  ['싶다']
A:  ['\t 방 여러 개인 집 에서 사세요 \n']
p:  ['지금 보다 더 잘 살 거 예요 \n']
_
Q:  ['있으면 좋겠다']
A:  ['\t 방 여러 개인 집 에서 사세요 \n']
p:  ['잘 \n']
_
Q:  ['딸기']
A:  ['\t 맛있게 드세요 \n']
p:  ['저 도 요 \n']
_
Q:  ['치고 싶다']
A:  ['\t 많이 힘든 가봅니다 \n']
p:  ['\n']
_
Q:  ['']
A:  ['\t 여행 좋죠 \n']
p:  ['\n']
_
Q:  ['또 개강 이라니']
A:  ['\t 즐거운 시작 되길 바랍니다 \n']
p:  ['방학 이 참 \n']
_
Q:  ['또 눈 내려']
A:  ['\t 눈 봐요 \n']
p:  ['내일 은 마음 도 거 예요 \n']
_
Q:  ['또 야근']
A:  ['\t 얼른 집 에 가서 바랄게요 \n']
p:  ['오늘 잠 이 이제 나 봐요 \n']
_
Q:  ['똑같은 들']
A:  ['\t 마음 에 따라 얼마 바뀔 수 있어요 \n']
p:  ['시간 이 무 거 죠 \n']
_
Q:  ['대로 한 거 같은데 왜 맛 이 없지']
A:  ['\t 이 안 그럴 수도 있어

_
Q:  ['번호 달라 고 해볼까']
A:  ['\t 용기 를 내 보세요 \n']
p:  ['말 해보세요 \n']
_
Q:  ['번호 볼까 나']
A:  ['\t 용기 를 내 보세요 \n']
p:  ['많이 지쳤나 봐요 \n']
_
Q:  ['벌써 개강 이라니']
A:  ['\t 방학 이 참 \n']
p:  ['방학 이 참 \n']
_
Q:  ['벌써 가 없어']
A:  ['\t 를 \n']
p:  ['에너지 가 필요한 순간 이네 요 \n']
_
Q:  ['법 을 피해 가는 사람']
A:  ['\t 능력 하지만 가 거 예요 \n']
p:  ['능력 하지만 가 거 예요 \n']
_
Q:  ['벚꽃 이 너무 예뻐']
A:  ['\t 너무 \n']
p:  ['부모님 이 \n']
_
Q:  ['베터리 야']
A:  ['\t 미리 충전 하세요 \n']
p:  ['미리 충전 하세요 \n']
_
Q:  ['에서 사업 해볼까']
A:  ['\t 좀 더 하세요 \n']
p:  ['확신 이 있을 때 시작 해보세요 \n']
_
Q:  ['이 걸 기다려']
A:  ['\t 기다리는 전화 가 있나 봐요 \n']
p:  ['상대 에게 피해 를 주는 건 어떨까 요 \n']
_
Q:  ['']
A:  ['\t 얼른 보아 요 \n']
p:  ['\n']
_
Q:  ['변화 를 좀 주고 싶다']
A:  ['\t 긍정 적 으로 바뀔 수 있어요 \n']
p:  ['많이 지쳤나 봐요 \n']
_
Q:  ['별 보러 가고 싶다']
A:  ['\t 로맨틱 하네요 \n']
p:  ['로맨틱 하네요 \n']
_
Q:  ['게 없는 일상']
A:  ['\t 그게 행복 일지 도 몰라요 \n']
p:  ['한 잔 하기 좋은 \n']
_
Q:  ['게 없는 하루']
A:  ['\t 마음 을 가져 보세요 \n']
p:  ['요즘 바쁜 가봐요 \n']
_
Q:  ['별 이 안']
A:  ['\t 에서 하늘 을 올려 봐 보세요 \n']
p:  ['\n']
_
Q:  ['병원 가기 싫어']
A:  ['\t 더 전 에

_
Q:  ['신발 선물 받았어']
A:  ['\t 건가 요 \n']
p:  ['말 해 주세요 \n']
_
Q:  ['준비 혼자 할 수 있을까']
A:  ['\t 신혼여행 은 이 죠 \n']
p:  ['혼자 만 있지 마세요 \n']
_
Q:  ['실내 데이트 좋을까']
A:  ['\t 이나 이 \n']
p:  ['야경 좋은 곳 으로 가보세요 \n']
_
Q:  ['실수 했다']
A:  ['\t 부족한 부분 이 뭔 지 확인 해보세요 \n']
p:  ['이제 \n']
_
Q:  ['뭐 하면 좋을까']
A:  ['\t 저 랑 이야기 해요 \n']
p:  ['평소 에 볼 수 없어요 \n']
_
Q:  ['']
A:  ['\t 저 랑 놀아요 \n']
p:  ['\n']
_
Q:  ['커 해볼까']
A:  ['\t 눈 은 기본 이 죠 \n']
p:  ['적 으로 도움 이 되겠네요 \n']
_
Q:  ['커 할까']
A:  ['\t 돈 많이 \n']
p:  ['\n']
_
Q:  ['이 나 만 해']
A:  ['\t 애정 표현 이라고 생각 하세요 \n']
p:  ['일 하고 있는데 연락 하기 쉽지 않죠 \n']
_
Q:  ['없는 걱정 1 위 연예인 걱정']
A:  ['\t 입니다 \n']
p:  ['그 분 생각 을 많이 했나 봐요 곧 좋은 소식 이 하는 것 아닐까 요 \n']
_
Q:  ['싫어']
A:  ['\t 푹 쉬세요 \n']
p:  ['상처 봐요 \n']
_
Q:  ['아 놔 왜']
A:  ['\t 자체 로 의미 가 있으니까 요 \n']
p:  ['그 분 생각 이 나나 봐요 \n']
_
Q:  ['아']
A:  ['\t 약 챙겨 치과 가보세요 \n']
p:  ['노래 불러 까요 \n']
_
Q:  ['아무 것 도 하고 싶지 아무 거나 하고 싶어']
A:  ['\t 위해 무엇 인가 를 하고 있는 게 좋을 거 예요 \n']
p:  ['저 도 요 \n']
_
Q:  ['아무래도 속 은 거 같아']
A:  ['\t 기분 \n']
p:  ['둘 이 아니라 일 이지 요 \

...

_
Q:  ['날 좋아하던 남자 애가 고백 이 없는데 맘 까']
A:  ['\t 속 마음 은 \n']
p:  ['정말 사랑 하 나 봅니다 \n']
_
Q:  ['남사 친 이 어떡해']
A:  ['\t 이상 이성 으로 \n']
p:  ['짝사랑 그 사람 을 위해 에너지 가 요 \n']
_
Q:  ['남사 친 이라 쓰고 짝남 이라']
A:  ['\t 슬픈 이야기 네 요 \n']
p:  ['에 보세요 \n']
_
Q:  ['남자 는 첫사랑 못 잊어']
A:  ['\t 첫사랑 못 \n']
p:  ['바쁜 걸 이해 못 하는 것 감 을 느껴 보세요 \n']
_
Q:  ['남자 들 직장 내 에서 좋아하는 여자 있으면 어떻게 해']
A:  ['\t 관심 을 표현 할 거 예요 \n']
p:  ['사랑 하는데 어떤 확신 이 \n']
_
Q:  ['남자 들 은 몇 년 씩 혼자 짝사랑 할 수도 있어']
A:  ['\t 계속 엄청 좋아하는 게 아니라 시간 이 거 죠 \n']
p:  ['정말 내 요 눈 을 못 되지 않을까요 \n']
_
Q:  ['남자 들 이 좋아하는 여자 들 은 뭐 야']
A:  ['\t 사람 마다 너무 다를 것 같아요 \n']
p:  ['저 랑 가슴 이 없어요 \n']
_
Q:  ['남자 랑 이 술 여자친구 를 이해 해줘 야할까']
A:  ['\t 이해 하기 거 같아요 \n']
p:  ['이해 할 수 없어요 \n']
_
Q:  ['남자친구 소개 자리 내 가 더 떨려']
A:  ['\t 남자친구 는 더 있을 거 예요 \n']
p:  ['적 으로 통보 하는 말 보다는 조금 더 하고 물어보는 말 을 해보세요 \n']
_
Q:  ['남자친구 이 성 친구 가 많아']
A:  ['\t 사귈 때 는 덜 말 해봐요 \n']
p:  ['친구 가 아닌 가봐요 \n']
_
Q:  ['남자친구 선물 로 뭐 가 좋을까']
A:  ['\t 직접 물어보는 게 편할 거 같아요 \n']
p:  ['평소 에 게 좋을 것 같아요 \n']
_
Q:  ['남자친구 가 sns 에 나 안 올려']
A:  

_
Q:  ['나 한테 다 남자친구 어떡하지']
A:  ['\t 걸 걱정 하면 아무 것 도 못 해요 \n']
p:  ['서로 가 전부 인 연애 가 아니라 서로 부족한 부분 을 서로 를 해주는 연애 인 것 같아요 \n']
_
Q:  ['미련 좋아하는 거 랑 뭐 가']
A:  ['\t 구분 어려운 것 같아요 \n']
p:  ['많이 알 지 못 했나 봐요 \n']
_
Q:  ['반 동거 중 인 여자친구 가 좋겠어']
A:  ['\t 자신 의 이 말 해보세요 \n']
p:  ['진심 으로 대하 고 싶다고 말 해보세요 \n']
_
Q:  ['밤 인데 되나']
A:  ['\t 를 구 해보세요 \n']
p:  ['전화 해보세요 \n']
_
Q:  ['배려 있는 사람 이 좋은데']
A:  ['\t 그런 사람 만나세요 \n']
p:  ['현실 의 벽 에 봐요 \n']
_
Q:  ['않는 사람 도 있겠지']
A:  ['\t 있을거예요 \n']
p:  ['저 도 매일 해요 \n']
_
Q:  ['별로 인 거 알면서도 좋아하는 내 가 이상해']
A:  ['\t 점 이 있다면 얼른 맘 을 좋아요 \n']
p:  ['아직 모르겠어요 \n']
_
Q:  ['보통 좋아하는 애 생기 면 먼저 연락 해']
A:  ['\t 은 먼저 연락 하 해 요 \n']
p:  ['정말 그래요 \n']
_
Q:  ['본인 을 좋아하는게 확실한 여자 와 친구 가 될 수 있어']
A:  ['\t 그 사람 을 위 그러면 안 돼요 \n']
p:  ['이성 으로 대해 요 \n']
_
Q:  ['부모님 께 드리고 싶은 사람 있어']
A:  ['\t 사랑 하는 사람인 가봐요 \n']
p:  ['\n']
_
Q:  ['부모님 이 이혼 거 같은데 나 는 어떻게 하지']
A:  ['\t 부모님 만의 이 있을거예요 이해 하려고 노력 해보세요 \n']
p:  ['평소 흘려 이야기 했던 걸 기억 해주고 생각 하고 있다는 걸 알려주세요 \n']
_
Q:  ['비 가 오 니까 짝사랑 하는 사람 더 생각나']
A:  ['\t 비 오는 

_
Q:  ['썸남 이 넘 가 안 돼']
A:  ['\t 자연스러운 현상 이에요 \n']
p:  ['썸 과 대화 하는 과정 이 가장 \n']
_
Q:  ['썸남 이랑 끝 인 것 같아 이 와']
A:  ['\t 슬픈 이네 요 \n']
p:  ['나쁜 생각 은 \n']
_
Q:  ['썸남 이랑 헬스장 에서 운동 가능']
A:  ['\t 따로 신경 부분 이 없다면 \n']
p:  ['시간 내서 가보세요 \n']
_
Q:  ['썸남 인데 가자고 함 어떻게 거절 해']
A:  ['\t 본인 의 의사 를 표현 하는 것 이 중요해요 \n']
p:  ['도움 이 얘기 해보세요 \n']
_
Q:  ['썸남 한테 같이 하자 고 할까']
A:  ['\t 좋죠 \n']
p:  ['진심 이 있을 수 있을 거 예요 \n']
_
Q:  ['썸남 한테 같이 카페 에서 공부 할까']
A:  ['\t 좋죠 \n']
p:  ['계획 을 보세요 \n']
_
Q:  ['썸남 한테 뭐라고 카톡 하지']
A:  ['\t 그냥 뭐 물어보세요 \n']
p:  ['직접 물어보는 게 좋겠어요 \n']
_
Q:  ['썸남 한테 책 선물 해도 괜찮을까']
A:  ['\t 기억 에 남 을 거 같아요 \n']
p:  ['꽃 을 마다 할 사람 은 없어요 \n']
_
Q:  ['썸녀 꿈 꿨어']
A:  ['\t 그 분 생각 을 많이 했나 봐요 곧 좋은 소식 이 하는 것 아닐까 요 \n']
p:  ['꿈 은 현실 과 반대 예요 \n']
_
Q:  ['썸녀 주변 에 남자 가 많아']
A:  ['\t 신경 \n']
p:  ['세상 에는 사람 들 이 있어요 \n']
_
Q:  ['썸녀 선물 뭐 가 좋을까']
A:  ['\t 부담 스럽지 않은 선 이 좋겠어요 \n']
p:  ['평소 에 필요한 것 생각 해보세요 \n']
_
Q:  ['썸녀 가']
A:  ['\t 축하 해주세요 \n']
p:  ['잘 하는 게 대요 \n']
_
Q:  ['썸녀 랑 거의 사귀는 인데 꼭 고백']
A:  ['\t 고백 을 안 하고 사귀는 경우 도 있

_
Q:  ['우리 반 여자애 가 나 좋아하는 거 너무 티']
A:  ['\t 생각 해보세요 \n']
p:  ['수도 있겠어요 \n']
_
Q:  ['']
A:  ['\t 소원 \n']
p:  ['\n']
_
Q:  ['이 사람 나를 좋아하는 건가']
A:  ['\t 상태 인가 봅니다 \n']
p:  ['아직 거 예요 \n']
_
Q:  ['이 사람 좋아하는 게']
A:  ['\t 고민 하고 있다면 맞는 거 아닐까 요 \n']
p:  ['자기 도 모르는 게 요 \n']
_
Q:  ['이 세상 에서 내 가 제일']
A:  ['\t 가요 \n']
p:  ['당신 을 하지 마세요 \n']
_
Q:  ['이 오빠 나를 좋아하는 건가']
A:  ['\t 만 알 거 예요 \n']
p:  ['감정 이 뿐 이에요 \n']
_
Q:  ['이 정도 면 내 가 좋아하는 걸']
A:  ['\t 확실한 가 \n']
p:  ['이제 알 수 있어요 \n']
_
Q:  ['이 거 뭘 까 좋아하는 걸까']
A:  ['\t 직접 적 인 마음 을 확인 하기 전 까지 확실한 건 \n']
p:  ['직접 물어보세요 \n']
_
Q:  ['이 거 썸 맞나']
A:  ['\t 대화 를 시작 하는 사람 이 누구 인지 살펴보세요 \n']
p:  ['적 인 수 있을 거 예요 \n']
_
Q:  ['이 거 썸']
A:  ['\t 말 해보세요 생각 해 \n']
p:  ['호감 이 있을 수도 있어요 그렇지만 조금 더 상황 을 지켜보세요 \n']
_
Q:  ['이 거 썸 인가요']
A:  ['\t 좋은 느낌 이면 썸 으로 만나 보세요 \n']
p:  ['썸 으로 사이 가 \n']
_
Q:  ['이 게 썸 인가']
A:  ['\t 말 해보세요 생각 해 \n']
p:  ['고민 하고 있으면 썸 의 가능성 이 \n']
_
Q:  ['이런 얼굴 좋아하는 사람 이 있을까요']
A:  ['\t 충분히 \n']
p:  ['행동 해 주세요 \n']
_
Q:  ['이렇게 사랑 되나']
A:  ['\t 행복한 고민 이네 요 \n'

_
Q:  ['짝남 여친 있는지 어떻게']
A:  ['\t 된 친구 한테 물어보세요 \n']
p:  ['서로 배려 하고 대화 하는 게 중요한 것 같아요 \n']
_
Q:  ['짝남 과 자연 스럽게 수 있는 방법 있을까']
A:  ['\t 관심사 를 찾아보세요 \n']
p:  ['거리 를 두세요 \n']
_
Q:  ['짝남 보다 더 나은 사람 은']
A:  ['\t 더 나은 사람 많아요 \n']
p:  ['매력 만큼 그런 사람 이네 요 \n']
_
Q:  ['짝남 이 갑자기 못 나']
A:  ['\t 어떤 가 거 예요 \n']
p:  ['sns \n']
_
Q:  ['짝남 이 날 신경 쓰는 것 같은 건 내 착각 일까']
A:  ['\t 좀 더 것 도 좋겠어요 \n']
p:  ['이상형 이 수도 있어요 \n']
_
Q:  ['짝남 이 날 좋아하게 할 방법 이 있을까']
A:  ['\t 우선 꾸준히 연락 하고 잘 해줘 보세요 \n']
p:  ['질투 를 연락 해보세요 \n']
_
Q:  ['짝남 이 내 친구 좋아하는 현실 어떻게']
A:  ['\t 가슴 인정 순간 이네 요 \n']
p:  ['너무 마세요 \n']
_
Q:  ['짝남 이 내 가 1 도']
A:  ['\t 1 은 알 게 해야죠 \n']
p:  ['나 사람 만나세요 \n']
_
Q:  ['짝남 이 너무 인기 가 많아서 같아 두려워']
A:  ['\t 얼른 적극 적 으로 \n']
p:  ['병원 에 연락 하고 관심 을 확인 해보세요 \n']
_
Q:  ['짝남 이 너무 좋은데 가 별로 야']
A:  ['\t 은 사귀고 나서 돼요 \n']
p:  ['직접 적 으로 물어보는게 좋겠어요 \n']
_
Q:  ['짝남 이 연락 이 안 되는데 자고 있는 걸까']
A:  ['\t 자고 몰라요 \n']
p:  ['상대방 을 헷갈리게 하는건 좋지 않아요 \n']
_
Q:  ['짝남 이 인데 연락 하면 귀찮아 할까']
A:  ['\t 가 되지 않는 시간 에 보내 보세요 \n']
p:  ['연락 은 싸움 의 이 죠 \n']


## Encoder with Attention

In [15]:
class EncoderA(tf.keras.Model):
    def __init__(self):
        super(EncoderA, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        
    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        H, h, c = self.lstm(x)
        return H, h, c

## Decoder with Attention

In [16]:
class DecoderA(tf.keras.Model):
    def __init__(self):
        super(DecoderA, self).__init__()
        # Adding layers for shifted output
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        # decoder need to store whole state from lstm so return_sequences should be True
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        self.att = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')
        
    def call(self, inputs, training=False, mask=None):
        y_, s0, c0, H = inputs
        S, h, c = self.lstm(y_, initial_state=[s0, c0])
        
        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)
        A = self.att([S_, H])
        y = tf.concat([S, A], axis=-1)
        
        return self.dense(y), h, c

## Seq2seq model with Attention

In [17]:
class Seq2seqA(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seqA, self).__init__()
        self.enc = EncoderA()
        self.dec = DecoderA()
        self.sos = sos
        self.eos = eos
        
    def call(self, inputs, training=False, mask=None):
        # When training, input is just shifted, so the process is very simple
        if training:
            x, y = inputs
            H, h, c = self.enc(x)
            y, _, _ = self.dec((y, h, c, H))
            return y
        else:
            x = inputs
            H, h, c = self.enc(x)
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))
            
            seq = tf.TensorArray(tf.int32, 64)
            
            # Through tf.range, we can iterate it with autoGraph
            for idx in tf.range(64):
                y, h, c = self.dec([y, h, c,H])
                
                # To convert sparse type
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                
                # To express batch shape
                y = tf.reshape(y, (1, 1))
                seq = seq.write(idx, y)
                
                if y == self.eos:
                    break
                
            return tf.reshape(seq.stack(), (1, 64))

In [18]:
# Create model
model = Seq2seqA(sos=tokenizer.word_index['\t'], eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [20]:
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]
    
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)
    
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

In [21]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1, 
                          train_loss.result(), 
                          train_accuracy.result() * 100))
    
    train_loss.reset_states()
    train_accuracy.reset_states()

ValueError: in user code:

    <ipython-input-20-9a717ee3f867>:7 train_step  *
        predictions = model([inputs, shifted_labels], training=True)
    <ipython-input-17-f6d2cfd8c3d4>:14 call  *
        y, _, _ = self.dec((y, h, c, H))
    <ipython-input-16-7f3ef2b274f4>:13 call  *
        S, h, c = self.lstm(y_, initial_state=[s0, c0])
    /home/chanseok/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/layers/recurrent.py:707 __call__  **
        return super(RNN, self).__call__(inputs, **kwargs)
    /home/chanseok/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:886 __call__
        self.name)
    /home/chanseok/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/input_spec.py:180 assert_input_compatibility
        str(x.shape.as_list()))

    ValueError: Input 0 of layer lstm_1 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [32, 64]
